# Word Vectors from Decomposing a Word-Word Pointwise Mutual Information Matrix
Lets create some simple [word vectors](https://en.wikipedia.org/wiki/Word_embedding) by applying a [singular value decomposition](https://en.wikipedia.org/wiki/Singular-value_decomposition) to a [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) word-word matrix.  There are many other ways to create word vectors, but matrix decomposition is one of the most straightforward.  A well cited description of the technique used in this notebook can be found in Chris Moody's blog post [Stop Using word2vec](https://multithreaded.stitchfix.com/blog/2017/10/18/stop-using-word2vec/).    If you are interested in reading further about the history of word embeddings and a discussion of modern approaches check out the following blog post by Sebastian Ruder, [An overview of word embeddings and their connection to distributional semantic models](http://blog.aylien.com/overview-word-embeddings-history-word2vec-cbow-glove/).  Especially interesting to me is the work by Omar Levy, Yoav Goldberg, and Ido Dagan which shows that tuning hyperparameters is as (if not more) important as the algorithm chosen to build word vectors. [Improving Distributional Similarity with Lessons Learned from Word Embeddings](https://transacl.org/ojs/index.php/tacl/article/view/570).

We will be using the ["A Million News Headlines"](https://www.kaggle.com/therohk/million-headlines) dataset which contains headlines published over a period of 15 years from the Australian Broadcasting Corporation (ABC).
It is a great clean corpus that is large enough to be interesting and small enough to allow for quick calculations.  In this notebook tutorial we will implement as much as we can without using libraries that obfuscate the algorithm.  We're not going to write our own linear algebra or sparse matrix routines, but we will calculate unigram frequency, skipgram frequency, and the pointwise mutual information matrix "by hand".  Hopefully this will make the method easier to understand!   

In [6]:
from collections import Counter
import itertools

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import linalg 
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

# Read Data and Preview

In [7]:
df = pd.read_csv('../input/abcnews-date-text.csv')
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


# Minimal Preprocessing
We're going to create a word-word co-occurrence matrix from the text in the headlines.  We will define two words as "co-occurring" if they appear in the same headline.  Using this definition, single word headlines are not interestintg for us.  Lets remove them as well as a common set of english stopwords.  

In [8]:
headlines = df['headline_text'].tolist()
# remove stopwords
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))
headlines = [
    [tok for tok in headline.split() if tok not in stopwords_set] for headline in headlines
]
# remove single word headlines
headlines = [hl for hl in headlines if len(hl) > 1]
# show results
headlines[0:20]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[['aba', 'decides', 'community', 'broadcasting', 'licence'],
 ['act', 'fire', 'witnesses', 'must', 'aware', 'defamation'],
 ['g', 'calls', 'infrastructure', 'protection', 'summit'],
 ['air', 'nz', 'staff', 'aust', 'strike', 'pay', 'rise'],
 ['air', 'nz', 'strike', 'affect', 'australian', 'travellers'],
 ['ambitious', 'olsson', 'wins', 'triple', 'jump'],
 ['antic', 'delighted', 'record', 'breaking', 'barca'],
 ['aussie', 'qualifier', 'stosur', 'wastes', 'four', 'memphis', 'match'],
 ['aust', 'addresses', 'un', 'security', 'council', 'iraq'],
 ['australia', 'locked', 'war', 'timetable', 'opp'],
 ['australia', 'contribute', '10', 'million', 'aid', 'iraq'],
 ['barca', 'take', 'record', 'robson', 'celebrates', 'birthday'],
 ['bathhouse', 'plans', 'move', 'ahead'],
 ['big', 'hopes', 'launceston', 'cycling', 'championship'],
 ['big', 'plan', 'boost', 'paroo', 'water', 'supplies'],
 ['blizzard', 'buries', 'united', 'states', 'bills'],
 ['brigadier', 'dismisses', 'reports', 'troops', 'harassed'

# Unigrams
Now lets calculate a unigram vocabulary.  The following code assigns a unique ID to each token, stores that mapping in two dictionaries (`tok2indx` and `indx2tok`), and counts how often each token appears in the corpus. 

In [9]:
unigram_counts = Counter()
for ii, headline in enumerate(headlines):
    if ii % 200000 == 0:
        print(f'finished {ii/len(headlines):.2%} of headlines')
    for token in headline:
        unigram_counts[token] += 1

tok2indx = {tok: indx for indx, tok in enumerate(unigram_counts.keys())}
indx2tok = {indx: tok for tok,indx in tok2indx.items()}
print('done')
print('vocabulary size: {}'.format(len(unigram_counts)))
print('most common: {}'.format(unigram_counts.most_common(10)))

finished 0.00% of headlines
finished 16.10% of headlines
finished 32.20% of headlines
finished 48.31% of headlines
finished 64.41% of headlines
finished 80.51% of headlines
finished 96.61% of headlines
done
vocabulary size: 117521
most common: [('police', 39812), ('new', 33729), ('man', 30627), ('says', 23731), ('us', 20370), ('court', 18179), ('govt', 17091), ('australia', 17071), ('council', 16955), ('nsw', 16516)]


# Skipgrams
Now lets calculate a skipgram vocabulary.  We will loop through each word in a headline (the focus word) and then form skipgrams by examing `back_window` words behind and `front_window` words in front of the focus word (the context words).  As an example, the first sentence (after preprocessing removes the stopword `against`) ,
```
aba decides community broadcasting licence
```
would produce the following skipgrams with `back_window`=`front_window`=`2`, 
```
('aba', 'decides')
('aba', 'community')
('decides', 'aba')
('decides', 'community')
('decides', 'broadcasting')
('community', 'aba')
('community', 'decides')
('community', 'broadcasting')
('community', 'licence')
('broadcasting', 'decides')
('broadcasting', 'community')
('broadcasting', 'licence')
('licence', 'community')
('licence', 'broadcasting')
```

In [10]:
# note add dynammic window hyperparameter

# note we store the token vocab indices in the skipgram counter
# note we use Levy, Goldberg, Dagan notation (word, context) as opposed to (focus, context)
back_window = 2
front_window = 2
skipgram_counts = Counter()
for iheadline, headline in enumerate(headlines):
    tokens = [tok2indx[tok] for tok in headline]
    for ii_word, word in enumerate(tokens):
        ii_context_min = max(0, ii_word - back_window)
        ii_context_max = min(len(headline) - 1, ii_word + front_window)
        ii_contexts = [
            ii for ii in range(ii_context_min, ii_context_max + 1) 
            if ii != ii_word]
        for ii_context in ii_contexts:
            skipgram = (tokens[ii_word], tokens[ii_context])
            skipgram_counts[skipgram] += 1    
    if iheadline % 200000 == 0:
        print(f'finished {iheadline/len(headlines):.2%} of headlines')
        
print('done')
print('number of skipgrams: {}'.format(len(skipgram_counts)))
most_common = [
    (indx2tok[sg[0][0]], indx2tok[sg[0][1]], sg[1]) 
    for sg in skipgram_counts.most_common(10)]
print('most common: {}'.format(most_common))

finished 0.00% of headlines
finished 16.10% of headlines
finished 32.20% of headlines
finished 48.31% of headlines
finished 64.41% of headlines
finished 80.51% of headlines
finished 96.61% of headlines
done
number of skipgrams: 8873036
most common: [('man', 'charged', 4017), ('charged', 'man', 4017), ('gold', 'coast', 3645), ('coast', 'gold', 3645), ('world', 'cup', 3320), ('cup', 'world', 3320), ('country', 'hour', 2971), ('hour', 'country', 2971), ('covid', '19', 2694), ('19', 'covid', 2694)]


# Sparse Matrices

We will calculate several matrices that store word-word information.  These matrices will be $N \times N$ where $N \approx 100,000$ is the size of our vocabulary.  We will need to use a sparse format so that it will fit into memory.  A nice implementation is available in [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).  To create these sparse matrices we create three iterables that store row indices, column indices, and data values. 

# Word-Word Count Matrix
Our very first word vectors will come from a word-word count matrix.  This matrix is symmetric so we can (equivalently) take the word vectors to be the rows or columns.  However we will try and code as if the rows are word vectors and the columns are context vectors. 

In [11]:
row_indxs = []
col_indxs = []
dat_values = []
ii = 0
for (tok1, tok2), sg_count in skipgram_counts.items():
    ii += 1
    if ii % 1000000 == 0:
        print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')    
    row_indxs.append(tok1)
    col_indxs.append(tok2)
    dat_values.append(sg_count)
wwcnt_mat = sparse.csr_matrix((dat_values, (row_indxs, col_indxs)))
print('done')

finished 11.27% of skipgrams
finished 22.54% of skipgrams
finished 33.81% of skipgrams
finished 45.08% of skipgrams
finished 56.35% of skipgrams
finished 67.62% of skipgrams
finished 78.89% of skipgrams
finished 90.16% of skipgrams
done


# Word Similarity with Sparse Count Matrices

In [12]:
def ww_sim(word, mat, topn=10):
    """Calculate topn most similar words to word"""
    indx = tok2indx[word]
    if isinstance(mat, sparse.csr_matrix):
        v1 = mat.getrow(indx)
    else:
        v1 = mat[indx:indx+1, :]
    sims = cosine_similarity(mat, v1).flatten()
    sindxs = np.argsort(-sims)
    sim_word_scores = [(indx2tok[sindx], sims[sindx]) for sindx in sindxs[0:topn]]
    return sim_word_scores
    

In [13]:
ww_sim('strike', wwcnt_mat)

[('strike', 1.0000000000000395),
 ('strikes', 0.5029895718322583),
 ('strike;', 0.45067772068232304),
 ('accept', 0.44915407340122415),
 ('striking', 0.447417012072658),
 ('underpaid', 0.4351099407643213),
 ('threaten', 0.43232112956503754),
 ('dispute', 0.42648873201955717),
 ('industrial', 0.4200996378604476),
 ('casual', 0.416335135299664)]

In [14]:
# normalize each row using L2 norm
wwcnt_norm_mat = normalize(wwcnt_mat, norm='l2', axis=1)

In [15]:
# demonstrate normalization
row = wwcnt_mat.getrow(10).toarray().flatten()
print(np.sqrt((row*row).sum()))

row = wwcnt_norm_mat.getrow(10).toarray().flatten()
print(np.sqrt((row*row).sum()))

198.11612756158948
1.0


In [16]:
ww_sim('strike', wwcnt_norm_mat)

[('strike', 0.9999999999999974),
 ('strikes', 0.5029895718322446),
 ('strike;', 0.45067772068231415),
 ('accept', 0.44915407340122215),
 ('striking', 0.4474170120726478),
 ('underpaid', 0.4351099407643136),
 ('threaten', 0.4323211295650257),
 ('dispute', 0.42648873201954407),
 ('industrial', 0.4200996378604436),
 ('casual', 0.41633513529965516)]

# Pointwise Mutual Information Matrices
The pointwise mutual information (PMI) for a (word, context) pair in our corpus is defined as the probability of their co-occurrence divided by the probabilities of them appearing individually, 
$$
{\rm pmi}(w, c) = \log \frac{p(w, c)}{p(w) p(c)}
$$

$$
p(w, c) = \frac{
f_{i,j}
}{
\sum_{i=1}^N \sum_{j=1}^N f_{i,j}
}, \quad 
p(w) = \frac{
\sum_{j=1}^N f_{i,j}
}{
\sum_{i=1}^N \sum_{j=1}^N f_{i,j}
}, \quad
p(c) = \frac{
\sum_{i=1}^N f_{i,j}
}{
\sum_{i=1}^N \sum_{j=1}^N f_{i,j}
}
$$
where $f_{i,j}$ is the word-word count matrix we defined above.
In addition we can define the positive pointwise mutual information as, 
$$
{\rm ppmi}(w, c) = {\rm max}\left[{\rm pmi(w,c)}, 0 \right]
$$

Note that the definition of PMI above implies that ${\rm pmi}(w, c) = {\rm pmi}(c, w)$ and so this matrix will be symmetric.  However this is not true for the variant in which we smooth over the contexts.

In [17]:
num_skipgrams = wwcnt_mat.sum()
assert(sum(skipgram_counts.values())==num_skipgrams)

# for creating sparce matrices
row_indxs = []
col_indxs = []

pmi_dat_values = []    # pointwise mutual information
ppmi_dat_values = []   # positive pointwise mutial information
spmi_dat_values = []   # smoothed pointwise mutual information
sppmi_dat_values = []  # smoothed positive pointwise mutual information

# reusable quantities

# sum_over_rows[ii] = sum_over_words[ii] = wwcnt_mat.getcol(ii).sum()
sum_over_words = np.array(wwcnt_mat.sum(axis=0)).flatten()
# sum_over_cols[ii] = sum_over_contexts[ii] = wwcnt_mat.getrow(ii).sum()
sum_over_contexts = np.array(wwcnt_mat.sum(axis=1)).flatten()

# smoothing
alpha = 0.75
sum_over_words_alpha = sum_over_words**alpha
nca_denom = np.sum(sum_over_words_alpha)



ii = 0
for (tok_word, tok_context), sg_count in skipgram_counts.items():

    ii += 1
    if ii % 1000000 == 0:
        print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')

    # here we have the following correspondance with Levy, Goldberg, Dagan
    #========================================================================
    #   num_skipgrams = |D|
    #   nwc = sg_count = #(w,c)
    #   Pwc = nwc / num_skipgrams = #(w,c) / |D|
    #   nw = sum_over_cols[tok_word]    = sum_over_contexts[tok_word] = #(w)
    #   Pw = nw / num_skipgrams = #(w) / |D|
    #   nc = sum_over_rows[tok_context] = sum_over_words[tok_context] = #(c)
    #   Pc = nc / num_skipgrams = #(c) / |D|
    #
    #   nca = sum_over_rows[tok_context]^alpha = sum_over_words[tok_context]^alpha = #(c)^alpha
    #   nca_denom = sum_{tok_content}( sum_over_words[tok_content]^alpha )
    
    nwc = sg_count
    Pwc = nwc / num_skipgrams
    nw = sum_over_contexts[tok_word]
    Pw = nw / num_skipgrams
    nc = sum_over_words[tok_context]
    Pc = nc / num_skipgrams
    
    nca = sum_over_words_alpha[tok_context]
    Pca = nca / nca_denom
    
    # note 
    # pmi = log {#(w,c) |D| / [#(w) #(c)]} 
    #     = log {nwc * num_skipgrams / [nw nc]}
    #     = log {P(w,c) / [P(w) P(c)]} 
    #     = log {Pwc / [Pw Pc]}
    pmi = np.log2(Pwc/(Pw*Pc))   
    ppmi = max(pmi, 0)
    spmi = np.log2(Pwc/(Pw*Pca))
    sppmi = max(spmi, 0)
    
    row_indxs.append(tok_word)
    col_indxs.append(tok_context)
    pmi_dat_values.append(pmi)
    ppmi_dat_values.append(ppmi)
    spmi_dat_values.append(spmi)
    sppmi_dat_values.append(sppmi)
        
pmi_mat = sparse.csr_matrix((pmi_dat_values, (row_indxs, col_indxs)))
ppmi_mat = sparse.csr_matrix((ppmi_dat_values, (row_indxs, col_indxs)))
spmi_mat = sparse.csr_matrix((spmi_dat_values, (row_indxs, col_indxs)))
sppmi_mat = sparse.csr_matrix((sppmi_dat_values, (row_indxs, col_indxs)))

print('done')

finished 11.27% of skipgrams
finished 22.54% of skipgrams
finished 33.81% of skipgrams
finished 45.08% of skipgrams
finished 56.35% of skipgrams
finished 67.62% of skipgrams
finished 78.89% of skipgrams
finished 90.16% of skipgrams
done


# Word Similarity with Sparse PMI Matrices

In [18]:
ww_sim('strike', pmi_mat)

[('strike', 0.9999999999999941),
 ('strikes', 0.1211889169729063),
 ('dispute', 0.111794737973725),
 ('industrial', 0.11013037404202804),
 ('workers', 0.10808509092483007),
 ('union', 0.09824258730931341),
 ('threaten', 0.09327705739895319),
 ('action', 0.09049153037571292),
 ('pay', 0.09037713413807796),
 ('protest', 0.08719699099636351)]

In [19]:
ww_sim('strike', ppmi_mat)

[('strike', 0.9999999999999979),
 ('strikes', 0.11957421585896628),
 ('industrial', 0.11420158772225265),
 ('dispute', 0.11407511639697168),
 ('workers', 0.10948499324430837),
 ('threaten', 0.09912745341712337),
 ('union', 0.09847871176787408),
 ('teachers', 0.09095151490694113),
 ('pay', 0.08713222969103766),
 ('action', 0.08589196247267036)]

In [20]:
ww_sim('strike', spmi_mat)

[('strike', 0.9999999999999989),
 ('strikes', 0.15379750195987027),
 ('industrial', 0.13868226499746528),
 ('dispute', 0.13281815348535653),
 ('threaten', 0.1143842771410373),
 ('workers', 0.11193625891405247),
 ('union', 0.10923253691286354),
 ('teachers', 0.10448961899575446),
 ('protest', 0.10388379644400357),
 ('irc', 0.10020335322179598)]

In [21]:
ww_sim('strike', sppmi_mat)

[('strike', 0.9999999999999994),
 ('strikes', 0.15880950542659844),
 ('industrial', 0.1500609679439667),
 ('dispute', 0.14359261813670962),
 ('workers', 0.13006771278240392),
 ('threaten', 0.1264820824467524),
 ('union', 0.12344283725083058),
 ('teachers', 0.11750624246563811),
 ('irc', 0.11406708556047618),
 ('protest', 0.11094191508660257)]

# Singular Value Decomposition
With the PMI and PPMI matrices in hand, we can apply a singular value decomposition to create dense word vectors from the sparse ones we've been using. 

In [22]:
pmi_use = ppmi_mat
embedding_size = 50
uu, ss, vv = linalg.svds(pmi_use, embedding_size) 

In [23]:
print('vocab size: {}'.format(len(unigram_counts)))
print('embedding size: {}'.format(embedding_size))
print('uu.shape: {}'.format(uu.shape))
print('ss.shape: {}'.format(ss.shape))
print('vv.shape: {}'.format(vv.shape))

vocab size: 117521
embedding size: 50
uu.shape: (117521, 50)
ss.shape: (50,)
vv.shape: (50, 117521)


In [24]:
unorm = uu / np.sqrt(np.sum(uu*uu, axis=1, keepdims=True))
vnorm = vv / np.sqrt(np.sum(vv*vv, axis=0, keepdims=True))
#word_vecs = unorm
#word_vecs = vnorm.T
word_vecs = uu + vv.T
word_vecs_norm = word_vecs / np.sqrt(np.sum(word_vecs*word_vecs, axis=1, keepdims=True))

In [25]:
def word_sim_report(word, sim_mat):
    sim_word_scores = ww_sim(word, word_vecs)
    for sim_word, sim_score in sim_word_scores:
        print(sim_word, sim_score)
        word_headlines = [hl for hl in headlines if sim_word in hl and word in hl][0:5]
        for headline in word_headlines:
            print(f'    {headline}')

In [26]:
word = 'strike'
word_sim_report(word, word_vecs)


strike 0.9999999999999999
    ['air', 'nz', 'staff', 'aust', 'strike', 'pay', 'rise']
    ['air', 'nz', 'strike', 'affect', 'australian', 'travellers']
    ['png', 'nurses', 'strike', 'colleague', 'raped', 'way']
    ['qantas', 'international', 'crews', 'strike', 'pay']
    ['brisbane', 'sparkies', 'head', '10', 'day', 'strike']
ankhmer 0.9742043773214719
    ['ankhmer', 'strike']
abkhaz 0.9608041671365147
    ['abkhaz', 'separatists', 'strike', 'disputed', 'georgia', 'gorge']
cnt 0.9462957849041833
    ['dendrobium', 'strike', 'cnt']
hreidarsson 0.9341210709024877
    ['hreidarsson', 'strike', 'piles', 'misery', 'rovers']
48hr 0.9052323116669418
    ['48hr', 'boeing', 'strike', 'go', 'ahead']
    ['riverina', 'teachers', 'threaten', '48hr', 'strike']
'unsanctioned' 0.9016168314938678
    ["'unsanctioned'", 'strike', 'halts', 'sydney', 'ferries']
reactivated 0.8605154618003175
    ['bushfire', 'arson', 'strike', 'force', 'reactivated']
mulgave 0.8602494067054247
    ['jeff', 'day', 'ho

In [27]:
word = 'war'
word_sim_report(word, word_vecs)

war 1.0
    ['australia', 'locked', 'war', 'timetable', 'opp']
    ['qantas', 'war', 'plan', 'cut', '2500', 'jobs', 'outrages', 'unions']
    ['saudi', 'arabia', 'tells', 'arabs', 'war', 'iraq', 'inevitable']
    ['fifa', 'boss', 'prepared', 'postpone', 'matches', 'war']
    ['howard', 'steps', 'criticism', 'anti', 'war', 'protesters']
duty' 0.9646419528226714
    ['ex', 'dictator', 'noriega', 'goes', 'war', "'call", "duty'"]
harneys 0.9632027533049581
    ['harneys', 'war', '1958']
preslections 0.9438787561314945
    ['labors', 'verbal', 'war', 'preslections']
ceranic 0.930076733728787
    ['irena', 'ceranic', 'fled', 'war', 'sarajevo', 'new', 'life', 'australia']
coriolanus 0.9264443245964926
    ['gerard', 'butler', 'war', 'coriolanus']
pk5 0.9203692427896819
    ['pope', 'calls', 'peace', 'christians;', 'muslims', 'war', 'torn', 'pk5']
aquired 0.9182945941839114
    ['hilda', 'rix', 'nicholas', 'war', 'sketches', 'aquired', 'awm']
'ideological' 0.9162069916176754
    ['amnesty', 'w

In [28]:
word = 'bank'
word_sim_report(word, word_vecs)

bank 1.0000000000000002
    ['commonwealth', 'bank', 'cuts', 'fixed', 'home', 'loan', 'rates']
    ['former', 'bank', 'worker', 'pleads', 'guilty', 'theft', 'charges']
    ['community', 'bank', 'shares', 'trading', 'shows', 'little', 'change']
    ['bank', 'launches', 'initiative', 'combat', 'identity', 'fraud']
    ['israel', 'arrests', '20', 'demolishes', 'house', 'west', 'bank']
explosive' 0.9587344440944157
    ['australian', 'delegation', 'west', 'bank', "'very", "explosive'"]
whio 0.9303038642157602
    ['whio', 'bank', 'cartel', 'six']
collapase 0.9223689389262507
    ['woman', 'hurt', 'bank', 'ceiling', 'collapase']
stateme 0.8847670599773373
    ['australian', 'dollar', 'hits', 'eight', 'month', 'high', 'reserve', 'bank', 'stateme']
dexia 0.8521116820836032
    ['deal', 'reached', 'dismantle', 'european', 'bank', 'dexia']
luci 0.8463483351975202
    ['reserve', 'bank', 'luci', 'ellis', 'asset', 'price', 'booms']
anz 0.8227994408508352
    ['anz', 'ditches', 'plans', 'thai', 'm

In [29]:
word = 'car'
word_sim_report(word, word_vecs)


car 0.9999999999999998
    ['surge', 'new', 'car', 'sales', 'aust', 'abs']
    ['sun', 'shines', 'rmit', 'entry', 'eco', 'car', 'race']
    ['police', 'ethics', 'unit', 'oversee', 'fatal', 'car', 'crash', 'probe']
    ['woman', 'dies', 'car', 'crash']
    ['man', 'dies', 'car', 'crash']
chses 0.9709010967684164
    ['car', 'chses']
klassic 0.9689220015142456
    ['klassic', 'transformations', 'car', 'club', 'mens', 'health', 'mental', 'suicide']
buzzcar 0.9686046271769343
    ['zipcar', 'buzzcar', 'car', 'car', 'sharing', 'latest']
zipcar 0.9684031816506744
    ['zipcar', 'buzzcar', 'car', 'car', 'sharing', 'latest']
bueller 0.967277656700281
    ['bueller', 'bueller', 'ferriss', 'car', 'sale']
culpirts 0.9662524682641749
    ['police', 'hunt', 'car', 'torching', 'culpirts']
waneroo 0.9513463819955511
    ['police', 'search', 'drivers', 'car', 'plowed', 'waneroo', 'h']
urus 0.9462677837090164
    ['teenager', 'crashes', 'stolen', 'car', 'lamborghini', 'urus', 'suv']
impoundment 0.94501

In [30]:
word = 'football'
word_sim_report(word, word_vecs)

football 1.0000000000000002
    ['matera', 'hangs', 'football', 'boots']
    ['crisis', 'german', 'football', 'despite', 'exit', 'dortmund']
    ['football', 'academy', 'seeks', 'expansion', 'funds']
    ['football', 'clubs', 'honour', 'bali', 'victim']
    ['accrington', 'stanley', 'one', 'step', 'away', 'football']
figaro 0.9719027766971854
    ['tau', 'pupua', 'football', 'figaro']
pupua 0.9710184916810043
    ['tau', 'pupua', 'football', 'figaro']
soccer 0.9165072934738359
    ['asa', 'dumps', 'soccer', 'football']
    ['football', 'funding', 'needed', 'host', 'soccer', 'stars']
    ['independiente', 'santa', 'fe', 'colombia', 'football', 'soccer', 'counterfeit']
    ['john', 'moriarty', 'football', 'program', 'australian', 'kids', 'world', 'cup', 'soccer']
    ['football', 'soccer', 'chelsea', 'fans', 'spark', 'new', 'racism', 'storm']
austus 0.906023358690396
    ['austus', 'football', 'blends', 'australian', 'american', 'gridiron', 'rules']
clean: 0.904630940277053
    ['footbal

In [31]:
word = 'tech'
word_sim_report(word, word_vecs)

tech 1.0000000000000002
    ['air', 'traffic', 'control', 'glitch', 'dents', 'japans', 'high', 'tech']
    ['school', 'air', 'go', 'high', 'tech']
    ['british', 'tornados', 'add', 'concrete', 'hi', 'tech', 'arsenal']
    ['german', 'bio', 'tech', 'company', 'sends', 'free', 'sars', 'test']
    ['high', 'tech', 'centre', 'shed', 'light', 'timber', 'industry']
syste 0.9229467501227471
    ['government', 'urged', 'expedite', 'compulsory', 'high', 'tech', 'braking', 'syste']
ishak 0.8653717204617998
    ['ultrasound', 'tech', 'samir', 'ishak', 'touched', 'genitals', 'breasts', 'court', 'hears']
homeamazon 0.8377082991578263
    ['google', 'homeamazon', 'tech', 'devices', 'threat', 'personal', 'data']
paule 0.8144077655283556
    ['pryor', 'paule', 'ag', 'tech']
smartfin 0.7804664141580837
startup 0.7774335300242273
    ['creative', 'tech', 'startups', 'face', 'judges', 'startup', 'weekend']
flagsi 0.7746060237481358
    ['queensland', 'beaches', 'go', 'high', 'tech', 'flagsi']
'resisting